In [1]:
import pandas as pd
import numpy as np
import copy
import itertools
# Package for Levenstein string distance
import jellyfish
import time

In [2]:
path = "C://Users//Dimitri//Desktop//ENSAE3A//NetworkData//Data//"

In [3]:
attrs = pd.read_csv(path + "attrs_sub.csv")

In [4]:
def authors_parser(authors_string, sep=";"):
    return authors_string.split("; ")

In [5]:
attrs["authors_list"] = attrs["authors"].apply(authors_parser)

In [6]:
def normalized_edit_distance(str1, str2):
    dist = jellyfish.levenshtein_distance(str1, str2)
    return dist / max(len(str1), len(str2))

In [9]:
# Fill all authors
authors = []
for author in attrs["authors_list"]:
    authors += author
# Remove duplicates
authors = list(set(authors))

In [16]:
authors

['Kwiek, Maksymilian',
 'George E. Barnett',
 'Dijk, Dick Van',
 'Spychalski, John C',
 'Matthew Serfling',
 'Litov, Lubomir',
 'Skrzypacz, Andrzej',
 'Paolo Pin',
 'Ashley, Richard A',
 'Hu, Wei-Yin',
 'James W. Albrecht',
 'Vicente Cunat',
 'Babus, Ana',
 'Madrian, Brigitte C',
 'Fitzgerald, Doireann',
 'Dungey, Mardi',
 'Saidenberg, Marc R.',
 'Vodopivec, Milan',
 'Attiat F. Ott',
 'Kilcollin, T. Eric',
 'Raman, Kartik',
 'Minardi, Stefania',
 'Clifton H. Keeps Jr.',
 'Jaroslav Borovička',
 'James R. Hines Jr.',
 'Charles R. Link',
 'Raymond Gradus',
 'Deza, Monica',
 'Roman Kozhan',
 'Dorsey Richardson',
 'Tarsalewska, Monika',
 'Joseph Gyourko',
 'I. L. Sharfman',
 'Lorenzo Codogno',
 'J. Douglas Brown',
 'Prüfer, Patricia',
 'G. M. Youssef',
 'Sjoquist, David L',
 ' Marshall, David A',
 'Schröder, Marcel',
 'Walker, Francis E.',
 'Henderson, J V',
 'Caroline Paunov',
 'Sean Mulholland',
 'Spiegel, M.',
 'Marie-Claire Villeval',
 'Östberg, Per',
 'Jeffrey Liebner',
 'Tomohiro Hira

In [14]:
df_authors = pd.DataFrame(authors, columns=["original"])

In [15]:
df_authors

,original
0,Matthijs R. Wildenbeest
1,"Campbell, John Y."
2,"Harrenstein, Paul"
3,J. Anthony Cookson
4,"Erard, Brian"
5,"Bradley, John"
6,Anmol Bhandari
7,"Train, Kenneth"
8,William T. Gibb
9,"Dehez, Pierre"


In [12]:
def uniformize_names(str1):
    if "," in str1:
        splitted = str1.split(", ")
        if len(splitted) > 1:
            str1bis = splitted[1] + " " + splitted[0]
        else :
            str1bis = str1
        return str1bis
    else:
        return str1

In [16]:
df_authors["uniformat"] = df_authors["original"].apply(uniformize_names)

In [47]:
df_authors_reduced = df_authors.iloc[11000: 12000, :].copy()

In [63]:
def map_authors(auths_df, thresh):
    cleaned_df = pd.DataFrame(columns=["original", "uniformat", "equivalent"])
    for i in auths_df.index:
        author = auths_df.loc[i, "uniformat"]
        author_original = auths_df.loc[i, "original"]
        splitted = author.split(" ")
        last = splitted[-1]
        search = cleaned_df["uniformat"].str.contains(last, regex=False)
        ind_search = search[search == True].index
        first = splitted[0]
        prop = (len(first) == 1) or ((len(first) == 2) and ("." in first))
        exists_similar = False
        for ind in ind_search :
            dist = normalized_edit_distance(cleaned_df["uniformat"][ind], 
                                            author)
            dist_test = dist <= thresh
            if dist_test :
                exists_similar = True
                cleaned_df["equivalent"][ind].append(author_original)    
            # first_letter_test1 = prop and (first[0] == cleaned_df["uniformat"][ind][0])
            # if first_letter_test1:
                # exists_similar = True
                # cleaned_df["equivalent"][ind].append(author_original) 
        if not exists_similar:
            df = pd.DataFrame(columns=["original", "uniformat", "equivalent"], index=[0])
            df.set_value(0, "original", author_original)
            df.set_value(0, "uniformat", author)
            df.set_value(0, "equivalent", [author_original])
            cleaned_df = cleaned_df.append(df, ignore_index=True)
        if i%1000 == 0:
            print(i)
    return cleaned_df

In [64]:
start = time.clock()
cleaned = map_authors(df_authors, 0.1)
end = time.clock()
print(end - start)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
1009.7924934654384


In [65]:
cleaned.to_csv(path + "authors.csv")